In [209]:
%%capture
import xarray as xr
import os
import time
import numpy as np
import gsw
import pyproj
import pandas as pd
from scipy.interpolate import interp1d

In [2]:
data = xr.open_dataset('/data/datos/ARGO/data/20181109_prof.nc')
data

<xarray.Dataset>
Dimensions:                       (N_CALIB: 1, N_HISTORY: 0, N_LEVELS: 1561, N_PARAM: 3, N_PROF: 215)
Dimensions without coordinates: N_CALIB, N_HISTORY, N_LEVELS, N_PARAM, N_PROF
Data variables:
    DATA_TYPE                     object ...
    FORMAT_VERSION                object ...
    HANDBOOK_VERSION              object ...
    REFERENCE_DATE_TIME           object ...
    DATE_CREATION                 object ...
    DATE_UPDATE                   object ...
    PLATFORM_NUMBER               (N_PROF) object ...
    PROJECT_NAME                  (N_PROF) object ...
    PI_NAME                       (N_PROF) object ...
    STATION_PARAMETERS            (N_PROF, N_PARAM) object ...
    CYCLE_NUMBER                  (N_PROF) float64 ...
    DIRECTION                     (N_PROF) object ...
    DATA_CENTRE                   (N_PROF) object ...
    DC_REFERENCE                  (N_PROF) object ...
    DATA_STATE_INDICATOR          (N_PROF) object ...
    DATA_MODE        

In [3]:
prof1=59
prof2=66
prof1 = data.sel(N_PROF=prof1)
prof2 = data.sel(N_PROF=prof2)

In [190]:
grid = np.arange(0,2001,10)
def current_v(data):
    temp = data.TEMP.data
    mask = ~np.isnan(temp)
    salt = data.PSAL.data
    pres = data.PRES.data
    lat, lon  = np.repeat(float(data.LATITUDE.data),grid.size), np.repeat(float(data.LONGITUDE.data),grid.size)
    depth = -gsw.conversions.z_from_p(pres[mask],np.repeat(lat[0],mask.size)[mask])
    
    func = interp1d(depth,temp[mask],fill_value='extrapolate')
    tmp_grid = np.where(grid>depth[-1],np.nan,grid) #(grid<depth[0]) | (grid>depth[-1])
    new_temp = func(tmp_grid)
    
    funcs = interp1d(depth,salt[mask],fill_value='extrapolate')
    new_salt = funcs(tmp_grid)
    new_pres = gsw.conversions.p_from_z(-grid,lat)
    new_pres[0]=0
    new_mask = ~np.isnan(new_temp)

    absalt = gsw.conversions.SA_from_SP(new_salt,new_pres,lon,lat)

    constpm = gsw.conversions.CT_from_t(absalt,new_temp,new_pres)
    svanom = gsw.specvol_anom_standard(absalt,constpm,new_pres)
    
    svanom_mean = (svanom[:-1]+svanom[1:])/2
    dpres = new_pres[1:]-new_pres[:-1]
    geop_anom = svanom_mean*dpres*10e4
    geop_anom[np.isnan(geop_anom)] = 0
    return geop_anom[::-1].cumsum()[::-1]

In [193]:
a = current_v(prof1)
b = current_v(prof2)
diff = a-b

In [204]:
lat1, lon1 = float(prof1.LATITUDE.data), float(prof1.LONGITUDE.data)
lat2, lon2 = float(prof2.LATITUDE.data), float(prof2.LONGITUDE.data)

In [207]:
lat1,lon1

(1.949, -100.087)

In [208]:
lat2,lon2

(1.05842, -104.53703)

In [205]:
geod = pyproj.Geod(ellps='WGS84')
azimuth1, azimuth2, distance = geod.inv(lon1, lat1, lon2, lat2)

In [206]:
distance

504897.13641989895

In [170]:
gsw.sigma0(gsw.conversions.SA_from_SP(33.71,0,41.91666667,-50.15),gsw.conversions.CT_from_t(gsw.conversions.SA_from_SP(33.71,0,41.91666667,-50.15),t=5.99,p=0))

26.536706085877768

In [28]:
gsw.sigma0(33.71,5.99)

26.412002768476214

In [32]:
gsw.specvol_anom_standard(gsw.conversions.SA_from_SP(33.71,0,41.91666667,-50.15),gsw.conversions.CT_from_t(gsw.conversions.SA_from_SP(33.71,0,41.91666667,-50.15),t=5.99,p=0),0)

1.4879012108148446e-06